# SageMaker VLLM endpoint example

## 1. Define some variables

The byoc will build and store a vllm endpoint docker image in you ECR private repo (for example `sagemaker_endpoint/vllm`), you need to define the following variables.

In [ ]:
MODEL_ID = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
INSTANCE_TYPE = "ml.g6e.12xlarge"
# better to work with vllm>=v0.7.2
VLLM_VERSION = "v0.7.2"
REPO_NAMESPACE = "sagemaker_endpoint/vllm"
ACCOUNT = !aws sts get-caller-identity --query Account --output text
REGION = !aws configure get region
ACCOUNT = ACCOUNT[0]
REGION = REGION[0]
if REGION.startswith("cn"):
    # this is a example repo port from vllm/vllm-openai, you can create your own docker image in your global region account
    VLLM_REPO = "public.ecr.aws/y0a9p9k0/vllm/vllm-openai"
    CONTAINER = f"{ACCOUNT}.dkr.ecr.{REGION}.amazonaws.com.cn/{REPO_NAMESPACE}:{VLLM_VERSION}"
else:
    VLLM_REPO = "vllm/vllm-openai"
    CONTAINER = f"{ACCOUNT}.dkr.ecr.{REGION}.amazonaws.com/{REPO_NAMESPACE}:{VLLM_VERSION}"

## 2. Build the container

Endpoint starting codes are in `app/`. The script will build and push to ecr. 

**The docker only need to be built once**, and after that, when deploying other endpoints, the same docker image can be shared.

In [ ]:
cmd = f"VLLM_REPO={VLLM_REPO} VLLM_VERSION={VLLM_VERSION} REPO_NAMESPACE={REPO_NAMESPACE} ACCOUNT={ACCOUNT} REGION={REGION} bash ./build_and_push.sh "
print("Runging:", cmd)
!{cmd}

## 3. Deploy on SageMaker

define the model and deploy on SageMaker


In [ ]:
%pip install -U boto3 sagemaker transformers huggingface_hub modelscope

### 3.1 Init SageMaker session

In [ ]:
import os
import re
import json
from datetime import datetime
import time

import boto3
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
default_bucket = sess.default_bucket()

sagemaker_client = boto3.client("sagemaker")

### 3.2 Download and upload model file

Firstly, you need to prepare model weights and upload to S3. You can download from HuggingFace, ModelScope or upload your own model. 

If you want vllm to automatically pull the model when it starts, this step can be skipped.

In [ ]:
model_name = MODEL_ID.replace("/", "-").replace(".", "-")
local_model_path = "./models/" + model_name
s3_model_path = f"s3://{default_bucket}/models/" + model_name

%mkdir -p code {local_model_path}

print("local_model_path:", local_model_path)

##### Option 1: Global region (download from HuggingFace)

In [ ]:
!huggingface-cli download --resume-download {MODEL_ID} --local-dir {local_model_path} --max-workers 32

##### Option 2: China region  (download from ModelScope)

In [ ]:
# !modelscope download --local_dir {local_model_path} {MODEL_ID} 

#### upload to s3

In [ ]:
!s5cmd sync --concurrency 32 {local_model_path}/ {s3_model_path}/
print("s3_model_path:", s3_model_path)

### 3.3 Prepare vllm start scripts

Then you need to a write the vllm starting scripts for endpoint, the container will automatically use the `start.sh` as the entrypont.

Please carefully modify the startup script file as needed, such as the model running parameter information. All parameters can be referenced at [https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html)

Here is a simple script that pulling a model from S3 and starting a vllm server.

In [ ]:
endpoint_model_name = sagemaker.utils.name_from_base(model_name, short=True)
local_code_path = endpoint_model_name
s3_code_path = f"s3://{default_bucket}/endpoint_code/vllm_byoc/{endpoint_model_name}.tar.gz"

%mkdir -p {local_code_path}

print("local_code_path:", local_code_path)

with open(f"{local_code_path}/start.sh", "w") as f:
    f.write(f"""
#!/bin/bash

# download model to local
s5cmd sync --concurrency 64 \
    {s3_model_path}/* /temp/model_weight


# the start script need to be adjust as you needed
# port needs to be $SAGEMAKER_BIND_TO_PORT

python3 -m vllm.entrypoints.openai.api_server \\
    --port $SAGEMAKER_BIND_TO_PORT \\
    --trust-remote-code \\
    --tensor-parallel-size 4 --max-model-len 65536 --enforce-eager \\
    --served-model-name {MODEL_ID} \\
    --model /temp/model_weight
""")

In [ ]:
!rm -f {local_code_path}.tar.gz
!tar czvf {local_code_path}.tar.gz {local_code_path}/
!aws s3 cp {local_code_path}.tar.gz {s3_code_path}
print("s3_code_path:", s3_code_path)

### 3.3 Deploy endpoint on SageMaker

In [ ]:
# Step 0. create model

# endpoint_model_name already defined in above step

create_model_response = sagemaker_client.create_model(
    ModelName=endpoint_model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": CONTAINER,
        "ModelDataUrl": s3_code_path
    },
    
)
print(create_model_response)
print("endpoint_model_name:", endpoint_model_name)

In [ ]:
# Step 1. create endpoint config

endpoint_config_name = sagemaker.utils.name_from_base(model_name, short=True)

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": endpoint_model_name,
            "InstanceType": INSTANCE_TYPE,
            "InitialInstanceCount": 1,
            "ContainerStartupHealthCheckTimeoutInSeconds": 1000,
            # "EnableSSMAccess": True,
        },
    ],
)
print(endpoint_config_response)
print("endpoint_config_name:", endpoint_config_name)

In [ ]:
# Step 2. create endpoint

endpoint_name = sagemaker.utils.name_from_base(model_name, short=True)

create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response)
print("endpoint_config_name:", endpoint_name)
while 1:
    status = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)["EndpointStatus"]
    if status != "Creating":
        break
    print(datetime.now().strftime('%Y%m%d-%H:%M:%S') + " status: " + status)
    time.sleep(60)
print("Endpoint created:", endpoint_name)

## 4. Test

You can invoke your model with SageMaker runtime.

In [ ]:
messages = [{
    "role": "user",
    "content": "Hi, who are you!"
}]

max_tokens = 4096

### 4.1 Message api non-stream mode

In [ ]:
sagemaker_runtime = boto3.client('runtime.sagemaker')

payload = {
    "model": MODEL_ID,
    "messages": messages,
    "max_tokens": max_tokens,
    "stream": False
}
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

print(json.loads(response['Body'].read())["choices"][0]["message"]["content"])

### 4.2 Message api stream mode

In [ ]:
payload = {
    "model": MODEL_ID,
    "messages": messages,
    "max_tokens": max_tokens,
    "stream": True
}

response = sagemaker_runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

buffer = ""
for t in response['Body']:
    buffer += t["PayloadPart"]["Bytes"].decode()
    last_idx = 0
    for match in re.finditer(r'^data:\s*(.+?)(\n\n)', buffer):
        try:
            data = json.loads(match.group(1).strip())
            last_idx = match.span()[1]
            print(data["choices"][0]["delta"]["content"], end="")
        except (json.JSONDecodeError, KeyError, IndexError) as e:
            pass
    buffer = buffer[last_idx:]
print()

### 4.3 Completion api non-stream mode

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_path, trust_remote_code=True)

prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

payload = {
    "model": MODEL_ID,
    "prompt": prompt,
    "max_tokens": max_tokens,
    "stream": False
}

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

print(json.loads(response['Body'].read())["choices"][0]["text"])

### 4.4 Completion api stream mode

In [ ]:
payload = {
    "model": MODEL_ID,
    "prompt": prompt,
    "max_tokens": max_tokens,
    "stream": True
}

response = sagemaker_runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

buffer = ""
for t in response['Body']:
    buffer += t["PayloadPart"]["Bytes"].decode()
    last_idx = 0
    for match in re.finditer(r'^data:\s*(.+?)(\n\n)', buffer):
        try:
            data = json.loads(match.group(1).strip())
            last_idx = match.end()
            # print(data)
            print(data["choices"][0]["text"], end="")
        except (json.JSONDecodeError, KeyError, IndexError) as e:
            pass
    buffer = buffer[last_idx:]
print()

### 4.5 Speed test

In [63]:
from transformers import AutoTokenizer

sagemaker_runtime = boto3.client('runtime.sagemaker')

messages = [{
    "role": "user",
    "content": "帮我写一首七言律诗介绍上海"
}]

payload = {
    "model": MODEL_ID,
    "messages": messages,
    "max_tokens": 4096,
    "temperature": 0.0,
    "stream": True,
    "stream_options": {"include_usage": True},
}

response = sagemaker_runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

buffer = ""
time_start = time.time()
first_token_latency = 0
output = []
for t in response['Body']:
    buffer += t["PayloadPart"]["Bytes"].decode()
    last_idx = 0
    for match in re.finditer(r'^data:\s*(.+?)(\n\n)', buffer):
        try:
            chunk = json.loads(match.group(1).strip())
            last_idx = match.span()[1]
            # print(chunk)
            if "usage" in chunk:
                print(chunk)
                input_tokens = chunk["usage"]["prompt_tokens"]
                output_tokens = chunk["usage"]["completion_tokens"]
            if "choices" in data and chunk["choices"][0]["delta"]["content"]:
                if first_token_latency == 0:
                    first_token_latency = time.time() - time_start
                output.append(chunk["choices"][0]["delta"]["content"])
                print(output[-1], end="", flush=True)
                
            


        except (json.JSONDecodeError, KeyError, IndexError) as e:
            pass
    buffer = buffer[last_idx:]


total_time = time.time() - time_start

print("\n" + "=" * 50)
print("Input_tokens", input_tokens, "Output_tokens", output_tokens)
print(f"First token latency {first_token_latency:.3} seconds")
print(f"Output speed {output_tokens/(total_time-first_token_latency):.3} tokens/seconds")
print("=" * 50)

好，用户让我帮他写一首七言律诗介绍上海。首先，我得理解七言律诗的结构，通常是八句，每句七个字，讲究平仄和对仗。然后，我要考虑上海的主要景观和象征，比如黄浦江、东方明珠、外滩等，这些都是上海的标志性景点。

接下来，我需要把这些元素融入诗中，同时保持语言的优美和韵律感。第一句可以从黄浦江开始，描述它的壮阔。第二句可以引入外滩，表现出历史与现代的结合。第三、四句可以描绘江上的景象，比如船只和灯光，增加动态感。

后面几句可以转向城市的繁华，比如金融中心和新天地，展示上海的多样性。最后两句要有一个总结，表达对上海的赞美和留恋。赏析部分需要解释诗中的意象和表达的情感，让读者更好地理解诗的内涵。

在写赏析时，要注意用词准确，突出诗中的亮点，比如对比手法和意象的运用。同时，要保持语言流畅，避免过于生硬。整体上，诗和赏析要相辅相成，共同展现上海的魅力。
</think>

《七律·沪上》
黄浦江头壮阔波，
外滩古色接天罗。
十里长街车水马，
万家灯火竹木桥。
金融才显峰峦险，
新天地开世界雕。
欲识沪城千面貌，
不离东珠一日遥。

赏析：这首作品以黄浦江、外滩等为背景，描绘了上海的繁华与壮阔。通过“十里长街车水马，万家灯火竹木桥”的描绘，生动展现了上海的都市景象。诗中“金融才显峰峦险，新天地开世界雕”的运用，对比手法巧妙，突出了上海的现代与传统。尾联则以东珠为引，表达了对上海的深深眷恋与赞美。{'id': 'chatcmpl-0c709e734ae3450c8f69d8a4736be233', 'object': 'chat.completion.chunk', 'created': 1739512712, 'model': 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B', 'choices': [], 'usage': {'prompt_tokens': 16, 'total_tokens': 534, 'completion_tokens': 518}}

Input_tokens 16 Output_tokens 518
First token latency 0.0737 seconds
Output speed 18.3 tokens/seconds
